In [1]:
from jupyterthemes import jtplot
jtplot.style(theme='chesterish')

In [2]:
import numpy as np
from PIL import Image, ImageDraw
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import math

In [3]:
"""
Corners Dictionary maps the every image to the bounding box coordinates: x0, y0, x1, y1
inner box for ball containment and outer box filter for background
"""
image_bounding_box = {
    "IMG_0854.jpg": [[879,1061,295,568], [131,316,998,1661]],
    "IMG_0855.jpg": [[127,669,846,1300], [973,1482,2,182]],
    "IMG_0856.jpg": [[381,747,888,1196], [986,1325,277,621]],
    "IMG_0857.jpg": [[349,577,748,1068], [926,1174,212,43]],
    "IMG_0858.jpg": [[339,630,681,1062], [1160,1110,169,19]],
    "IMG_0859.jpg": [[383,670,816,1170], [928,1250,11,1]],
    "IMG_0860.jpg": [[833,639,1383,1232],[238,426,1492,1319]],
    "IMG_0861.jpg": [[965,611,1373,1049],[1496,1128,817,505]],
    "IMG_0862.jpg": [[926,712,1336,1124],[1434,1217,803,615]],
    "IMG_0863.jpg": [[1190,1088,610,438],[1318,1197,482,105]],
}
data_dir = "data/image_segmentation_dataset/"

In [11]:
def draw_rectangle(img_path, rect_coordinates):
    """
    Rect co-ordinates in [x0, y0, x1, y1]
    """
    im = Image.open(img_path)
    draw = ImageDraw.Draw(im)
    draw.rectangle(rect_coordinates)
    im.show()
    
def get_dataset(data_dir):
    """
    Returns:
        colors_box        : N1x3 RGB data within bounding box
        colors_background : N2x3 RGB data for background
    """
    colors_box = []
    colors_background = []
    for img_name, boxes in tqdm(image_bounding_box.items()):
        image_path = os.path.join(data_dir, img_name)
        img = Image.open(image_path)
        values = img.load()
        bbox, obox = boxes[0], boxes[1]
        for i in range(img.size[0]): #width x-coordinates
            for j in range(img.size[1]): #height y-coordinates
                if i >= bbox[0] and i <= bbox[2] and j >= bbox[1] and j <= bbox[3]:
                    colors_box.append(values[i, j])
                elif i < obox[0] or i > obox[2] or j < obox[1] or j > obox[3]:
                    colors_background.append(values[i, j])
                    
    return np.asarray(colors_box), np.asarray(colors_background)


In [12]:
# Generate the dataset for computation
colors_box, colors_bg = get_dataset(data_dir)
print(colors_box.shape, colors_bg.shape)

100%|██████████| 10/10 [00:13<00:00,  1.62s/it]


(1922981, 3) (23674751, 3)


In [15]:
# Optimize this function if memory errors or taking too long
def covariance_over_N(values):
    V = np.array([np.outer(x, x) for x in values])
    V = np.sum(V, axis=0)
    V = V/len(values)
    return V

In [16]:
# CAREFUL BEFORE RERUNNING THIS: TAKES LONG.
# Find mean and variance for colors_bg and colors_box
mean_box, mean_bg = np.mean(colors_box, axis=0), np.mean(colors_bg, axis=0)
covariance_box, covariance_bg = covariance_over_N(colors_box), covariance_over_N(colors_bg)

In [17]:
# On Test Time, we need to call only this function
def get_likelihood(c, mean, covariance):
    c_minus_mean = c - mean
    exp_value = math.exp((-1./2)*(c_minus_mean.T.dot(np.linalg.inv(covariance)).dot(c_minus_mean)))
    likelihood = (1./(math.pow(math.pi*2,3/2)*math.pow(np.linalg.det(covariance), 1/2)))*exp_value
    return likelihood
    
def predict(c, mean_ball, covariance_ball, mean_bg, covariance_bg):
    """
    Given an RGB pixel value, predict wether the color is of the ball or not.
    This is currentl based on the assumption that the ball color is a Gaussian random
    variable with the estimated mean and covariance provided in the parameters.
    Params:
        c               : 3x1 vector of color values of a pixel
        mean_ball       : 3x1 vector of mean ball color
        covariance_ball : 3x3 matrix of covariance of ball
        mean_bg         : 3x1 vector of mean background color
        covariance_bg   : 3x3 matrix of covariance of background
    Returns:
        is_ball : True if ball else False
    """
    likelihood_ball = get_likelihood(c, mean_ball, covariance_ball)
    likelihood_bg = get_likelihood(c, mean_bg, covariance_bg)
    neuman_test = likelihood_ball/likelihood_bg
    return True if neuman_test >= 1 else False

In [20]:
c = [187, 156, 117]
print(mean_box, covariance_box)
print(mean_bg, covariance_bg)
predict(c, mean_ball=mean_box, mean_bg=mean_bg, covariance_ball=covariance_box, covariance_bg=covariance_bg)

[187.58283935 156.1258728  117.95517012] [[39340.40140802 33155.97923484 25370.5380178 ]
 [33155.97923484 28149.30596974 21693.02878812]
 [25370.5380178  21693.02878812 16974.38468659]]
[80.29653723 71.74433163 61.85751267] [[11899.60616528 10398.10917268  8562.98092664]
 [10398.10917268  9312.12330085  7913.48221753]
 [ 8562.98092664  7913.48221753  7078.66372778]]


True

In [19]:
im = Image.open(os.path.join(data_dir, "IMG_0862.jpg"))
pixels = im.load() # create the pixel map
for i in range(im.size[0]): # for every pixel:
    for j in range(im.size[1]):
        if predict(c=pixels[i,j], mean_ball=mean_box, mean_bg=mean_bg, covariance_ball=covariance_box, covariance_bg=covariance_bg):
            pixels[i,j] = (0, 0 ,0)
im.show()
